# **Mid-Price Prediction and Feature Importance Evaluation Across Forecasting Models and Industries**



### Author: Nicholas Ffinch  
### University of Edinburgh Business School

*Disclaimer - Copilot and ChatGPT were used to debug this code*

# Access Google drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Conversion to CSV File
This paper uses the *Benchmark Dataset for Mid-Price Forecasting of Limit Order Book Data with Machine Learning Methods* by Adam Ntakaris et al. (2018) available below:

https://etsin.fairdata.fi/dataset/73eb48d7-4dbc-4a10-a52a-da745b47a649

Run the following code to convert the .txt into .csv file

In [ ]:
def conver_data():
    data_train_a = pd.read_csv('/content/drive/MyDrive/Train_Dst_NoAuction_ZScore_CF_1.txt', delimiter = r'\s+', header=None)
    data_train_a.to_csv('/content/drive/MyDrive/Dissertation/Zscore/train_no_auc.csv', sep=';', index=False)

    data_train = pd.read_csv('/content/drive/MyDrive/Train_Dst_Auction_ZScore_CF_1.txt', delimiter = r'\s+', header=None)
    data_train.to_csv('/content/drive/MyDrive/Dissertation/Zscore/train.csv', sep=';', index=False)

    data_test = pd.read_csv('/content/drive/MyDrive/Test_Dst_Auction_ZScore_CF_1.txt', delimiter = r'\s+', header=None)
    data_test.to_csv('/content/drive/MyDrive/Dissertation/Zscore/test.csv', sep=';', index=False)
    return

# Install all the required packages

In [ ]:
%pip install -r requirements.txt

# Run all imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import silhouette_score, mean_squared_error, r2_score, accuracy_score, f1_score, cohen_kappa_score, precision_score, roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Activation, Add, Conv1D, Conv2D, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from scipy.special import gamma
import scipy.signal as signal
from scipy.signal import savgol_filter
from statsmodels.tsa.arima.model import ARIMA
from IPython.display import display
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import shap
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, precision_score
import shutil
import os
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from sklearn.model_selection import TimeSeriesSplit
import warnings


# **Part 1: Preprocessing and Data Preparation**



# Split data into the 5 stocks


### **Stock Segmentation Based on Price Jumps**  

This script processes **Z score normalized financial market data** that contains information on **five different stocks**. To extract each individual stock, we identify the **start and end indices** using a segmentation approach based on **the highest mid-price jumps**, as recommended by **Adamantios Ntakaris**, one of the dataset authors.



### **Methodology:**
1. **Data Normalization**  
   - The dataset is already **normalized**, ensuring consistency across features and reducing scale-related distortions.


2. **Stock Segmentation Using Price Jumps**  
   - The mid-price series is analysed to compute **absolute price changes** (using `np.diff()`).
   - The **five largest mid-price jumps** are identified to determine the **most significant segmentation points**.
   - These jump indices are used as **boundaries** to extract **five distinct stock datasets**.

3. **Training and Test Set Splitting**  
   - The dataset (`data`) is divided into **five stock segments** based on the identified segmentation points.


In [ ]:
def get_stocks_old(data, MID_PRICE_INDEX = 41, OUTLIERS = 20):
  # Extract mid-price data from data_df
  mid_prices = data.iloc[:, MID_PRICE_INDEX].values  # Using row 41 from data

  # Compute absolute differences in mid-price
  mid_price_changes = np.diff(mid_prices)

  # Find the indices of the 5 largest changes
  largest_changes_indices = np.argsort(mid_price_changes)[-5:]

  # Sort indices for readability
  largest_changes_indices = np.sort(largest_changes_indices)

  # Ensure exactly 5 segments by adding the last index
  split_indices = np.concatenate(([0], largest_changes_indices, [len(mid_prices)]))

  # Keep only the first 6 split indices (to form exactly 5 segments)
  split_indices = split_indices[:6]  # Ensures exactly 5 segments

  print("Final Split Indices:", split_indices)

  # Create a dictionary to store the 5 stock segments
  split_data = {}

  for i in range(len(split_indices) - 1):
      start, end = split_indices[i], split_indices[i + 1]
      split_data[f'stock_{i+1}'] = mid_prices[start:end]

  # Convert to DataFrame for better visualization
  split_df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in split_data.items()]))

  # Split the Training data matrix into 5 segments based on `split_indices`
  data_stock_1 = data.iloc[split_indices[0]:split_indices[1], :]
  data_stock_2 = data.iloc[split_indices[1]:split_indices[2], :]
  data_stock_3 = data.iloc[split_indices[2]:split_indices[3], :]
  data_stock_4 = data.iloc[split_indices[3]:split_indices[4], :]
  data_stock_5 = data.iloc[split_indices[4]:, :]  # Ensure the last segment extends to the end

  return [data_stock_1, data_stock_2, data_stock_3, data_stock_4, data_stock_5, data]


# Feature Engineering for Mid-Price Prediction

The code below implements a range of feature engineering functions that will be called upon later

## Rolling Window Utility

- `rolling_sum(x, window)`: Computes a rolling sum over a specified window.

## Price-Based Features

- `get_returns_mid_price`: Calculates mid-price returns (log or simple).
- `get_realized_variance`: Cumulative realized variance.
- `get_realized_variance_rolling`: Rolling version of realized variance.

## Limit Order Book (LOB) Features

- `get_BOOK_imbalance`: Computes volume imbalance across LOB levels.
- `get_FLOW_imbalance`: Measures flow imbalance (volume changes).
- `get_market_depth`: Total market depth at specified LOB levels.

## Volatility and Noise Measures

- `get_bipower_variation` / `get_bipower_variation_rolling`: Variance using absolute return products.
- `get_rolling_std`: Rolling standard deviation.
- `get_noise_variance_MP` / `get_noise_variance_MP_rolling`: Microstructure noise variance.
- `get_noise_variance_returns` / `_rolling`: Return-based noise estimator.

## Advanced Variance Measures

- `get_RQ` / `get_RQ_rolling`: Realized quarticity.
- `get_RQT` / `get_RQT_rolling`: Tripower quarticity estimator.
- `get_realised_kernel`: Robust volatility estimator using autocovariation and Parzen kernel.

## Smoothing

- `get_ema`: Exponential moving average of price series.

> Note: Most rolling versions use a default window size of 30 unless otherwise specified.


In [ ]:
WINDOW = 30


# Helper function for rolling sum with a given window size.
def rolling_sum(x, window):
    result = np.empty(len(x))
    for i in range(len(x)):
        start = max(0, i - window + 1)
        result[i] = np.sum(x[start:i+1])
    return result



#####################################################################################
#                            Feature Engineering Functions                          #
#####################################################################################

# Define the features
def get_returns_mid_price(mid_price, use_log_returns = False):
    if use_log_returns:
        return np.diff(np.log(mid_price))

    return np.diff(mid_price)

################################################################################

def get_realized_variance(returns, use_log_returns=False):

    # Calculate the realised variance as the sum of squared returns
    rv = np.empty(returns.size+1)
    rv[0] = np.nan
    rv[1:] = np.cumsum(returns ** 2)
    return rv

################################################################################

# Modified realised variance: rolling sum of squared returns over a window of 10 events.
def get_realized_variance_rolling(returns, window=10, use_log_returns=False):
    rv = np.empty(returns.size + 1)
    rv[0] = np.nan
    # Instead of a cumulative sum over all returns, compute the rolling sum.
    rv[1:] = rolling_sum(returns ** 2, window)
    return rv

################################################################################

#TODO
def get_BOOK_imbalance(data_matrix, level = 5):
    # Establish indexes in the data
    ask_volumes = np.zeros(data_matrix.shape[1])
    bid_volumes = np.zeros(data_matrix.shape[1])
    for i in range(level):
        index_1 = 1 + 4 * i
        index_2 = 3 + 4 * i
        ask_volumes += data_matrix[index_1, :]
        bid_volumes += data_matrix[index_2, :]

    denominator = ask_volumes + bid_volumes
    weighted_imbalance = np.where(denominator != 0,(ask_volumes - bid_volumes) / (ask_volumes + bid_volumes),0)
    return weighted_imbalance

################################################################################

#TODO
def get_FLOW_imbalance(data_matrix, level = 1):
    # Establish indexes in the data
    index_1 = 1 + 4 * (level - 1)
    index_2 = 3 + 4 * (level - 1)
    ask_volumes = data_matrix[index_1, :]
    bid_volumes = data_matrix[index_2, :]

    # Get the previous bid and ask volumes
    bid_volumes_prev = np.roll(bid_volumes, 1)
    ask_volumes_prev = np.roll(ask_volumes, 1)

    # Calculate the imbalance
    imbalance = ((bid_volumes - bid_volumes_prev) - (ask_volumes - ask_volumes_prev))
    return imbalance

################################################################################

def get_market_depth(data_matrix, level = 5):
    depth = np.zeros(data_matrix.shape[1])
    for i in range(level):
        index_1 = 1 + 4 * i
        index_2 = 3 + 4 * i
        depth += data_matrix[index_1, :] + data_matrix[index_2, :]
    return depth

################################################################################

# Realised Kernel Calculation
def get_parzen_kernel(x):
    """Parzen kernel function for noise-robust QV estimation."""
    abs_x = np.abs(x)
    if abs_x <= 0.5:
        return 1 - 6 * (abs_x ** 2) + 6 * (abs_x ** 3)
    elif abs_x <= 1:
        return 2 * ((1 - abs_x) ** 3)
    return 0  # Zero outside the range

################################################################################

def get_autocovariation(X, h):
    """Compute autocovariation at lag h."""
    if h == 0:
        return np.var(X, ddof=1)  # Variance at lag 0
    return np.mean((X[:-h] - np.mean(X)) * (X[h:] - np.mean(X)))  # Covariance at lag h

################################################################################

def get_realised_kernel(X, H=10):
    N = len(X)
    RK = np.zeros(N)

    for t in range(N):
        # Compute the lag-0 autocovariation using data up to time t.
        gamma_0 = get_autocovariation(X[:t+1], 0)
        kernel_sum = 0.0

        # For each lag h, ensure we have enough observations (i.e., t >= h)
        for h in range(1, min(H, t) + 1):
            # Weight for the h-th lag from the Parzen kernel function.
            weight = get_parzen_kernel(h / H)
            # Compute the autocovariations at lag h and -h using data up to t.
            gamma_h = get_autocovariation(X[:t+1], h)
            gamma_minus_h = get_autocovariation(X[:t+1], -h)
            kernel_sum += weight * (gamma_h + gamma_minus_h)

        # The cumulative realized kernel at time t.
        RK[t] = gamma_0 + kernel_sum

    return RK


################################################################################

def get_preaveraged_realized_variance(prices, returns, delta_n, theta=1):

    n = len(prices)
    # Determine pre-averaging window size H (must be at least 2)
    H = max(int(np.floor(theta * np.sqrt(n))), 2)

    # Define the pre-averaging function g(x) = min(x, 1-x) for x in [0,1]
    def g(x):
        return min(x, 1 - x)

    # Compute the sum of squared pre-averaged increments
    preavg_squared_sum = 0.0
    # The index i runs from 0 to n - H + 1 (inclusive)
    for i in range(n - H + 2):
        preavg = 0.0
        for j in range(1, H):
            preavg += g(j / H) * returns[i + j - 1]
        preavg_squared_sum += preavg ** 2

    # Set psi values for g(x) = min(x, 1-x)
    psi1 = 1.0
    psi2 = 1.0 / 12.0

    # Sum of squared returns (the realized variance component)
    sum_squared_returns = np.sum(returns ** 2)

    # Compute the pre-averaged realized variance (PA-RV)
    PA_RV = (np.sqrt(delta_n) / (theta * psi2)) * preavg_squared_sum - \
            (psi1 * delta_n / (2 * theta**2 * psi2)) * sum_squared_returns

    return PA_RV

################################################################################

def get_bipower_variation(r):

    # Compute the product of consecutive absolute returns:
    # For i=1,...,len(r)-1, prod[i-1] = |r[i]| * |r[i-1]|

    prod = np.abs(r[1:]) * np.abs(r[:-1])
    cum_prod = np.cumsum(prod)

    # Allocate an array for BV with the same length as prices
    BV = np.full(r.shape, np.nan)
    # For t >= 2 (i.e. price index 2 corresponds to the first full pair), assign:
    return np.array((np.pi / 2) * cum_prod)

# Modified bipower variation: rolling sum of products of consecutive absolute returns.
def get_bipower_variation_rolling(r, window=WINDOW):
    # Calculate element-wise product for consecutive absolute returns.
    prod = np.abs(r[1:]) * np.abs(r[:-1])
    # Compute the rolling sum over the window.
    rolling_prod = rolling_sum(prod, window)
    # Multiply by the constant.
    # Note: The length of rolling_prod is the same as prod; you may want to pad to match r.
    BV = np.full(len(r), np.nan)
    # We assign the rolling bipower variation starting from index 1 (or adjust as needed)
    BV[1:len(rolling_prod)+1] = (np.pi / 2) * rolling_prod
    return BV

################################################################################

def get_rolling_std(returns, window = WINDOW):

    n = len(returns)
    rolling_std = np.full(n, np.nan)

    # For each time t where we have a full window, compute the rolling standard deviation.
    for t in range(0, n):
        if n-t-window == 0:
            window -= 1
        window_returns = returns[t:t + window]
        mean_return = np.mean(window_returns)
        # Note: Using 1/N as in the provided formula (i.e. the population standard deviation).
        rolling_std[t] = np.sqrt(np.mean((window_returns - mean_return) ** 2))

    return rolling_std

################################################################################
def get_noise_variance_MP (returns):
    n = len(returns)
    prod = returns[1:] * returns[:-1]
    return(-1/(n-1) * np.cumsum(prod))

# Modified noise variance (MP): rolling sum version.
def get_noise_variance_MP_rolling(returns, window=WINDOW):
    # Compute product of consecutive returns.
    prod = returns[1:] * returns[:-1]
    # Rolling sum over the product.
    rolling_prod = rolling_sum(prod, window)
    # Instead of dividing by (n-1), you might normalize by (window-1) to reflect the window length.
    return -1 / (window - 1) * rolling_prod
################################################################################

def get_noise_variance_returns (returns):
    n = len(returns)
    return(1/(2*n) * np.cumsum(returns**2))

# Modified noise variance based on returns: rolling sum version.
def get_noise_variance_returns_rolling(returns, window=WINDOW):
    rolling_sq = rolling_sum(returns ** 2, window)
    return 1 / (2 * window) * rolling_sq

################################################################################

def get_RQ(returns):
    return(len(returns)/3 *np.cumsum(returns**4))

# Modified RQ: rolling sum of fourth powers.
def get_RQ_rolling(returns, window=WINDOW):
    # Rolling sum of r^4 over the window.
    rolling_r4 = rolling_sum(returns ** 4, window)
    # Normalize similarly to your original (originally len(returns)/3, now window/3)
    return (window / 3) * rolling_r4

################################################################################
def get_RQT(returns):

    n = len(returns)
    if n < 3:
        raise ValueError("Return series must contain at least 3 observations.")

    # Compute μ₄/₃ = 2^(2/3) * Γ(7/6) / sqrt(pi)
    mu_4_3 = 2**(2/3) * gamma(7/6) / np.sqrt(np.pi)

    # Compute |r|^(4/3) for each return
    r_power = np.abs(returns) ** (4/3)

    # For i = 3, ..., n (1-indexed), the product term is:
    # |r_i|^(4/3) * |r_{i-1}|^(4/3) * |r_{i-2}|^(4/3)
    # In Python (0-indexed), sum over indices i = 2 to n-1:
    product_terms = r_power[2:] * r_power[1:-1] * r_power[:-2]

    sum_product = np.cumsum(product_terms)

    RQ_t = n * (mu_4_3 ** -3) * sum_product
    return RQ_t

# Modified RQT: rolling sum of products of three consecutive |r|^(4/3)
def get_RQT_rolling(returns, window=WINDOW):
    n = len(returns)
    if n < 3:
        raise ValueError("Return series must contain at least 3 observations.")
    # Constant from your original code.
    mu_4_3 = 2**(2/3) * gamma(7/6) / np.sqrt(np.pi)
    r_power = np.abs(returns) ** (4/3)
    # Compute product terms for triple consecutive observations.
    product_terms = r_power[:-2] * r_power[1:-1] * r_power[2:]
    # Compute the rolling sum over these product terms.
    rolling_prod = rolling_sum(product_terms, window)
    return window * (mu_4_3 ** -3) * rolling_prod

################################################################################

def get_ema(prices, period = 10):

    n = len(prices)
    if n == 0:
        return np.array([])

    alpha = 2 / (period + 1)
    ema = np.empty(n)
    ema[:period] = np.mean(prices[:period]) #Mean of the first period

    for t in range(1, n):
        ema[t] = alpha * prices[t] + (1 - alpha) * ema[t - 1]

    return ema

## **Feature Extraction for Market Data Analysis**
This script defines the function **`get_features(fdata, level=5)`**, which extracts a wide range of features from market data for high-frequency trading analysis. It processes **limit order book (LOB) data** and computes various financial metrics.

#### **Key Operations:**
1. **Data Transformation**  
   - Converts the input dataframe into a NumPy array for faster computations.

2. **Mid-Price Calculation**  
   - Extracts mid-prices at five depth levels.

3. **Bid-Ask Spread Calculation**  
   - Computes bid-ask spreads across five depth levels.

4. **Return Calculations**  
   - Derives log returns for mid-prices.

5. **Price Differences Across Levels**  
   - Computes price differences between adjacent bid and ask levels.

6. **Statistical Features**  
   - Calculates mean prices and volumes for bid/ask sides.  
   - Computes accumulated price and volume differences.

7. **Time-Sensitive Features**  
   - Computes price and volume derivatives over time.  
   - Extracts trade intensity and limit activity acceleration.

8. **Order Flow & Market Dynamics**  
   - Computes **Order Flow Imbalance (OFI)** and **Order Book Imbalance (OBI)**.  
   - Measures **Market Depth**.  

9. **Advanced Financial Metrics**  
   - Computes **Realized Variance, Kernel Variance, Bipower Variance, Jump Variation, and Rolling Standard Deviation**.  
   - Estimates **Noise Variance** for mid-price and returns.  
   - Computes **Exponential Moving Averages (EMA)** and **Realized Quadratic Variations (RQ & RQT)**.

10. **Feature Dictionary Output**  
   - Returns a structured dictionary containing all extracted features for further analysis.

In [ ]:

#####################################################################################
#                                Feature Building                                   #
#####################################################################################

def get_features(fdata, level = 5):
    # create a matrix of the data
    data_matrix = fdata.to_numpy()


    # Calculate the mid price
    mid_price_l1 = data_matrix[41, :]
    mid_price_l2 = data_matrix[43, :]
    mid_price_l3 = data_matrix[45, :]
    mid_price_l4 = data_matrix[47, :]
    mid_price_l5 = data_matrix[49, :]

    # Calculate bid ask spread
    spread_l1 = data_matrix[40, :]
    spread_l2 = data_matrix[42, :]
    spread_l3 = data_matrix[44, :]
    spread_l4 = data_matrix[46, :]
    spread_l5 = data_matrix[48, :]


    # Get returns for the midprice assumed log return
    mid_price_returns_l1 = get_returns_mid_price(mid_price_l1)
    mid_price_returns_l2 = get_returns_mid_price(mid_price_l2)
    mid_price_returns_l3 = get_returns_mid_price(mid_price_l3)
    mid_price_returns_l4 = get_returns_mid_price(mid_price_l4)
    mid_price_returns_l5 = get_returns_mid_price(mid_price_l5)

    # Get Price differences from data set

    Price_diff_ask_l1_l2 = data_matrix[62,:]
    Price_diff_ask_l2_l3 = data_matrix[64,:]
    Price_diff_ask_l3_l4 = data_matrix[66,:]
    Price_diff_ask_l4_l5 = data_matrix[68,:]
    Price_diff_ask_l1_l5 = data_matrix[16,:] - data_matrix[0,:]

    Price_diff_bid_l1_l2 = data_matrix[63,:]
    Price_diff_bid_l2_l3 = data_matrix[65,:]
    Price_diff_bid_l3_l4 = data_matrix[67,:]
    Price_diff_bid_l4_l5 = data_matrix[69,:]
    Price_diff_bid_l1_l5 = data_matrix[2,:] - data_matrix[19,:]


    # Get Price and Volume Means
    Price_mean_ask = data_matrix[80,:]
    Price_mean_bid = data_matrix[81,:]
    Vol_mean_ask = data_matrix[82,:]
    Vol_mean_bid = data_matrix[83,:]

    # Accumulated differences
    accumulated_diff_price = data_matrix[84,:]
    accumulated_diff_volume = data_matrix[85,:]

    ############################################################################
    #                      EXTRACT Time Insensitive DATA                       #
    ############################################################################

    # Extract derivation over dt

    # Derivation on price dP
    dP_ask_l1 = data_matrix[86,:]
    dP_ask_l2 = data_matrix[90,:]
    dP_ask_l3 = data_matrix[94,:]
    dP_ask_l4 = data_matrix[98,:]
    dP_ask_l5 = data_matrix[102,:]

    dP_bid_l1 = data_matrix[87,:]
    dP_bid_l2 = data_matrix[91,:]
    dP_bid_l3 = data_matrix[95,:]
    dP_bid_l4 = data_matrix[99,:]
    dP_bid_l5 = data_matrix[103,:]

    # Derivation on volume dV
    dV_ask_l1 = data_matrix[88,:]
    dV_ask_l2 = data_matrix[92,:]
    dV_ask_l3 = data_matrix[96,:]
    dV_ask_l4 = data_matrix[100,:]
    dV_ask_l5 = data_matrix[104,:]

    dV_bid_l1 = data_matrix[89,:]
    dV_bid_l2 = data_matrix[93,:]
    dV_bid_l3 = data_matrix[97,:]
    dV_bid_l4 = data_matrix[101,:]
    dV_bid_l5 = data_matrix[105,:]


    # Extract Average Intensity per type - 6 types
    intensity_1 = data_matrix[126,:]
    intensity_2 = data_matrix[127,:]
    intensity_3 = data_matrix[128,:]
    intensity_4 = data_matrix[129,:]
    intensity_5 = data_matrix[130,:]
    intensity_6 = data_matrix[131,:]

    # Extract Relative Intensity Comparison - 6 types
    intensity_comp_1 = data_matrix[132,:]
    intensity_comp_2 = data_matrix[133,:]
    intensity_comp_3 = data_matrix[134,:]
    intensity_comp_4 = data_matrix[135,:]
    intensity_comp_5 = data_matrix[136,:]
    intensity_comp_6 = data_matrix[137,:]

    # Extract Limit Activity Acceleration - 6 types
    LAA_1 = data_matrix[138,:]
    LAA_2 = data_matrix[139,:]
    LAA_3 = data_matrix[140,:]
    LAA_4 = data_matrix[141,:]
    LAA_5 = data_matrix[142,:]
    LAA_6 = data_matrix[143,:]

    # Get Price Movement
    # -1 because we count from 0
    movement_1 = data_matrix[144,:] -1 # t+1
    movement_5 = data_matrix[147,:] -1 # t+5
    movement_6 = data_matrix[148,:] -1 # t+10

    #Realised Variance
    realised_variance = get_realized_variance_rolling(mid_price_returns_l1)

    # Calculate the the order flow imbalance

    OFI = get_FLOW_imbalance(data_matrix)[1:]

    # Calculate the the order flow imbalance
    OBI = get_BOOK_imbalance(data_matrix)[1:]

    # # Calculate Market Depth
    Market_depth = get_market_depth(data_matrix)[1:]

    # # Calculate the financial duration
    # #times = message_matrix[:, 0]
    # #financial_duration = np.diff(times)

    # # Average mid price financial duration
    # #AMPD = financial_duration * mid_prices[1:]

    # # Cancelation rate calculation
    # #cancelation_rate = get_cancellation_ratio(message_matrix)[1:]

    # Calculate the realized kernel
    realised_kernel_mid_prices = get_realised_kernel(mid_price_l1)[1:]

    # # Calculate preaveraged realized variance
    # PRV = get_preaveraged_realized_variance(mid_price_l1, mid_price_returns_l1, 1)
    # # Calculate the exponential moving average
    ema_mid_prices = get_ema(mid_price_l1, 10)

    # # Calculate Bipower Variance
    BV = get_bipower_variation_rolling(mid_price_returns_l1)
    # print(BV.shape)

    # # Calculate Jump Variation
    JV = realised_variance[1:] - BV

    # print(realised_variance.shape)
    # # Calculate Rolling Standard D  eviation
    R_std_dev = get_rolling_std(mid_price_returns_l1, 0)

    NV_MP = get_noise_variance_MP_rolling(mid_price_returns_l1)
    NV_returns = get_noise_variance_returns_rolling(mid_price_returns_l1)
    RQ = get_RQ_rolling(mid_price_returns_l1)
    RQT = get_RQT_rolling(mid_price_returns_l1)


    return {
    # Features extracted from data

    ########################################################################
    #                          U2 Time-Insensitive                         #
    ########################################################################

    # Mid prices
    'mid_prices': np.array(mid_price_l1[3:]),
    'mid_prices_level_2': np.array(mid_price_l2[3:]),
    'mid_prices_level_3': np.array(mid_price_l3[3:]),
    'mid_prices_level_4': np.array(mid_price_l4[3:]),
    'mid_prices_level_5': np.array(mid_price_l5[3:]),

    ##Spreads
    'spread_level_1': np.array(spread_l1[3:]),
    'spread_level_2': np.array(spread_l2[3:]),
    'spread_level_3': np.array(spread_l3[3:]),
    'spread_level_4': np.array(spread_l4[3:]),
    'spread_level_5': np.array(spread_l5[3:]),

    ## Mid price returns
    'mid_price_returns_level_1': np.array(mid_price_returns_l1[2:]),
    'mid_price_returns_level_2': np.array(mid_price_returns_l2[2:]),
    'mid_price_returns_level_3': np.array(mid_price_returns_l3[2:]),
    'mid_price_returns_level_4': np.array(mid_price_returns_l4[2:]),
    'mid_price_returns_level_5': np.array(mid_price_returns_l5[2:]),

    ## Price difference between levels
    # ASK
    'Price_diff_ask_l1_l2': np.array(Price_diff_ask_l1_l2[3:]),
    'Price_diff_ask_l2_l3': np.array(Price_diff_ask_l2_l3[3:]),
    'Price_diff_ask_l3_l4': np.array(Price_diff_ask_l3_l4[3:]),
    'Price_diff_ask_l4_l5': np.array(Price_diff_ask_l4_l5[3:]),
    'Price_diff_ask_l1_l5': np.array(Price_diff_ask_l1_l5[3:]),

    ## BID
    'Price_diff_bid_l1_l2': np.array(Price_diff_bid_l1_l2[3:]),
    'Price_diff_bid_l2_l3': np.array(Price_diff_bid_l2_l3[3:]),
    'Price_diff_bid_l3_l4': np.array(Price_diff_bid_l3_l4[3:]),
    'Price_diff_bid_l4_l5': np.array(Price_diff_bid_l4_l5[3:]),
    'Price_diff_bid_l1_l5': np.array(Price_diff_bid_l1_l5[3:]),

    # Get Price and Volume Means
    'Price_mean_ask': np.array(Price_mean_ask[3:]),
    'Price_mean_bid': np.array(Price_mean_bid[3:]),
    'Vol_mean_ask': np.array(Vol_mean_ask[3:]),
    'Vol_mean_bid': np.array(Vol_mean_bid[3:]),

    ## Accumulated differences
    'accumulated_diff_price': np.array(accumulated_diff_price[3:]),
    'accumulated_diff_volume': np.array(accumulated_diff_volume[3:]),

    ########################################################################
    #                           U3 Time-Sensitive                          #
    ########################################################################

    ## Derivation on price dP and volume dV
    #ASK - Price
    'dP_ask_l1': np.array(dP_ask_l1[3:]),
    'dP_ask_l2': np.array(dP_ask_l2[3:]),
    'dP_ask_l3': np.array(dP_ask_l3[3:]),
    'dP_ask_l4': np.array(dP_ask_l4[3:]),
    'dP_ask_l5': np.array(dP_ask_l5[3:]),

    #BID - Price
    'dP_bid_l1': np.array(dP_bid_l1[3:]),
    'dP_bid_l2': np.array(dP_bid_l2[3:]),
    'dP_bid_l3': np.array(dP_bid_l3[3:]),
    'dP_bid_l4': np.array(dP_bid_l4[3:]),
    'dP_bid_l5': np.array(dP_bid_l5[3:]),

    #ASK - Volume
    'dV_ask_l1': np.array(dV_ask_l1[3:]),
    'dV_ask_l2': np.array(dV_ask_l2[3:]),
    'dV_ask_l3': np.array(dV_ask_l3[3:]),
    'dV_ask_l4': np.array(dV_ask_l4[3:]),
    'dV_ask_l5': np.array(dV_ask_l5[3:]),

    #BID - Volume
    'dV_bid_l1': np.array(dV_bid_l1[3:]),
    'dV_bid_l2': np.array(dV_bid_l2[3:]),
    'dV_bid_l3': np.array(dV_bid_l3[3:]),
    'dV_bid_l4': np.array(dV_bid_l4[3:]),
    'dV_bid_l5': np.array(dV_bid_l5[3:]),

    ## Extract Average Intensity per type - 6 types
    'intensity_1': np.array(intensity_1[3:]),
    'intensity_2': np.array(intensity_2[3:]),
    'intensity_3': np.array(intensity_3[3:]),
    'intensity_4': np.array(intensity_4[3:]),
    'intensity_5': np.array(intensity_5[3:]),
    'intensity_6': np.array(intensity_6[3:]),

    ## Extract Relative Intensity Comparison - 6 types
    'intensity_comp_1': np.array(intensity_comp_1[3:]),
    'intensity_comp_2': np.array(intensity_comp_2[3:]),
    'intensity_comp_3': np.array(intensity_comp_3[3:]),
    'intensity_comp_4': np.array(intensity_comp_4[3:]),
    'intensity_comp_5': np.array(intensity_comp_5[3:]),
    'intensity_comp_6': np.array(intensity_comp_6[3:]),

    # Extract Limit Activity Acceleration - 6 types
    'LAA_1': np.array(LAA_1[3:]),
    'LAA_2': np.array(LAA_2[3:]),
    'LAA_3': np.array(LAA_3[3:]),
    'LAA_4': np.array(LAA_4[3:]),
    'LAA_5': np.array(LAA_5[3:]),
    'LAA_6': np.array(LAA_6[3:]),

    ########################################################################
    #                           Computed features                          #
    ########################################################################

    'realised_variance': np.array(realised_variance[3:]),
    'Order Flow Imbalance': np.array(OFI[2:]),
    'Order Book Imbalance': np.array(OBI[2:]),
    'Market Depth': np.array(Market_depth[2:]),
    'Realised Kernel': np.array(realised_kernel_mid_prices[2:]),
    # 'Preaverage Realised variance': np.array(PRV)
    'Exponential Moving Average': np.array(ema_mid_prices[3:]),
    'Bipower Variance': np.array(BV[2:]),
    'Jump Variation': np.array(JV[2:]),
    # 'Rolling Standard Deviation': np.array(R_std_dev[2:]),
    'Noise Variance MP': np.array(NV_MP[1:]),
    'Noise Variance Returns': np.array(NV_returns[2:]),
    'RQ': np.array(RQ[2:]),
    'RQT': np.array(RQT),
    'Price Movement t+1': np.array(movement_1[3:]),
    'Price Movement t+5': np.array(movement_5[3:]),
    'Price Movement t+10': np.array(movement_6[3:])
    }





# Data Visualization and Diagnostics

This script includes functions for visualizing mid-price series, analyzing correlation between features, and computing multicollinearity using VIF.

### `plot_stocks`
Generates and saves smoothed mid-price plots for multiple stocks using moving averages. Each stock is plotted individually and saved as a PNG image.

### `plot_corr_matrix`
Creates and saves correlation matrix heatmaps for each stock dataset. Depending on the Correlated flag, output is stored in different subfolders.

### `save_vif_analysis`
Computes Variance Inflation Factor (VIF) for each stock's features to assess multicollinearity. Results are saved as CSV files.




In [ ]:
def plot_stocks(dfs, path, smoothing_window= 300, OUTLIERS = 50):

    # Define file path
    file_path = os.path.join(path, 'Mid_Price_Graph')

    # Define a moving average smoothing function.
    def smooth_series(series, window=smoothing_window):
        return np.convolve(series, np.ones(window) / window, mode='valid')

    num_stocks = len(dfs)  # Adjust if needed

    # Ensure the target directory exists.
    os.makedirs(file_path, exist_ok=True)

    # Loop over each stock.
    for i, df in enumerate(dfs):
        # Assume OUTLIERS is a global constant defined elsewhere
        stock_data = df['mid_prices'][OUTLIERS:-OUTLIERS]
        smoothed_stock = smooth_series(stock_data)

        i += 1
        # Create a new figure for each stock.
        plt.figure(figsize=(12, 6))
        plt.plot(smooth_series(stock_data, 20), label=f'Mid Price Stock {i}', color='blue')
        plt.plot(smoothed_stock, label=f'Moving Average Stock {i}', color='red')
        plt.xlabel('Events')
        plt.ylabel('Mid-Price')
        plt.title(f'Plot for the Stock #{i} mid-price')
        plt.legend()
        plt.grid(True)

        # Save the figure with a unique name.
        utput_file = os.path.join(file_path, f"Plot_for_the_{i}_mid_price.png")
        plt.savefig(output_file)
        plt.close()

        print(f"Plot saved to: {output_file}")

    # Example usage (assuming split_data dictionary and OUTLIERS constant are defined):
    # plot_and_save_each_stock_mid_price(split_data)
    return

def plot_corr_matrix(dfs, path, Correlated = False):

    if Correlated == True:
        file_path = os.path.join(path, 'Correlation_Matrix_Clean')
    else:
        file_path = os.path.join(path, 'Correlation_Matrix_No_Corr')
    # Loop over each DataFrame in the list and save its correlation matrix plot

    for i, df in enumerate(dfs, start=1):
        # Compute the correlation matrix
        corr_matrix = df.corr()

        # Create a figure for the heatmap
        plt.figure(figsize=(12, 8))
        sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', fmt='.2f', linewidths=0.5)
        plt.title(f'Correlation Matrix of Dataset {i}')

        # Create the full file path
        output_file = os.path.join(file_path, f"correlation_matrix_stock_{i}.png")

        # Save the plot to the file
        plt.savefig(output_file)
        plt.close()

        print(f"Plot saved to: {output_file}")

    return

def save_vif_analysis(feature_df_No_Corr, path):
    file_path = os.path.join(path, 'VIF')

    # Loop over each DataFrame in the list
    for i, df in enumerate(feature_df_No_Corr, start=1):
        # Optionally, select only numeric columns (if needed)
        X = sm.add_constant(df)

        # Create a DataFrame to hold the VIF results
        vif_data = pd.DataFrame()
        vif_data["Feature"] = X.columns
        vif_data["VIF"] = [variance_inflation_factor(X.values, j)
                           for j in range(X.shape[1])]

        # Define the output CSV file path
        csv_file = os.path.join(file_path, f"VIF_stock_{i}.csv")

        # Save the VIF results to CSV
        vif_data.to_csv(csv_file, index=False)
        print(f"VIF analysis for dataset {i} saved to: {csv_file}")

    return

# Data Extraction
In this section, we invoke the *get_features* function to extract the necessary features for the next phase.

Our objective is to extract features for both the training and test datasets.

As part of the analysis, six datasets are generated for both training and testing, each containing features for:

1. The first stock  
2. The second stock  
3. The third stock  
4. The fourth stock  
5. The fifth stock  
6. All stocks combined

\
Thereafter, we convert the data into dataframes.


\
**N.B:**
*There is no need to normalise the data as it has already been preprocessed*

In [ ]:
def get_feature_dfs_data(train_data, test_data):


    # Get features for all 5 training stocks and make data frame
    features_df_1 = pd.DataFrame(get_features(train_data[0].T))
    features_df_2 = pd.DataFrame(get_features(train_data[1].T))
    features_df_3 = pd.DataFrame(get_features(train_data[2].T))
    features_df_4 = pd.DataFrame(get_features(train_data[3].T))
    features_df_5 = pd.DataFrame(get_features(train_data[4].T))
    features_df_all = pd.DataFrame(get_features(train_data[5].T))


    # Get features for all 5 testing stocks and make data frame
    Test_features_df = pd.DataFrame(get_features(test_data[0].T))
    Test_features_df_2 = pd.DataFrame(get_features(test_data[1].T))
    Test_features_df_3 = pd.DataFrame(get_features(test_data[2].T))
    Test_features_df_4 = pd.DataFrame(get_features(test_data[3].T))
    Test_features_df_5 = pd.DataFrame(get_features(test_data[4].T))
    Test_features_df_all = pd.DataFrame(get_features(test_data[5].T))


    # List of feature DataFrames
    feature_dfs = [features_df_1, features_df_2, features_df_3, features_df_4, features_df_5, features_df_all]
    Test_features_dfs = [Test_features_df, Test_features_df_2, Test_features_df_3, Test_features_df_4, Test_features_df_5, Test_features_df_all]

    return feature_dfs, Test_features_dfs


## Clean Data Function

Removes zero-variance features (i.e., features that contain only zeros) from both training and testing feature datasets.

### Functionality:
- Scans the first DataFrame in the training set to identify features with only 0.0 values.
- Excludes specific target columns from this check: `'Price Movement t+1'`, `'Price Movement t+5'`, and `'Price Movement t+10'`.
- Drops identified zero-variance features from all DataFrames in both the training and testing sets.
- Prints the list of removed features for transparency.

### Returns:
- `feature_dfs_clean`: Cleaned list of training feature DataFrames.
- `Test_features_dfs_clean`: Cleaned list of testing feature DataFrames.


In [ ]:
def clean_data(features_dfs, Test_features_dfs):
      # Feature DataFrames (Replace with your actual DataFrames)
      feature_dfs_clean = features_dfs.copy()
      Test_features_dfs_clean = Test_features_dfs.copy()

      feature_df_clean = features_dfs[0].copy()

      # Identify features that contain only 0.0 values

      # Exclude specific columns from zero variance check
      exclude_columns = ['Price Movement t+1', 'Price Movement t+5', 'Price Movement t+10']

      # Identify zero-variance features, excluding specified columns
      zero_variance_features = [
      col for col in feature_df_clean.columns
      if col not in exclude_columns  # Exclude these columns
      and feature_df_clean[col].nunique() == 1
      and feature_df_clean[col].iloc[0] == 0.0
      ]


      # Store removed zero-variance features
      removed_zero_variance_features = zero_variance_features.copy()

      # Apply zero-variance feature removal to lists of feature DataFrames
      for df in feature_dfs_clean:
          df.drop(columns=zero_variance_features, errors='ignore', inplace=True)

      for df in Test_features_dfs_clean:
          df.drop(columns=zero_variance_features, errors='ignore', inplace=True)

      # Print removed features
      print("Removed zero-variance features:", removed_zero_variance_features)

      return feature_dfs_clean, Test_features_dfs_clean


## Correlated Features Removal - `remove_corr_values`

Removes highly correlated features from the training and testing datasets based on a specified correlation threshold.

### Functionality:
- Computes the correlation matrix of the first training DataFrame.
- Identifies feature pairs with absolute correlation above `MAX_CORR` (default: 0.7), excluding self-correlations.
- Retains one feature from each highly correlated pair and removes the other, unless the feature is one of the essential columns:
  `'mid_prices'`, `'Price Movement t+1'`, `'Price Movement t+5'`, `'Price Movement t+10'`.
- Applies the same feature removal to both training and testing datasets.

### Parameters:
- `feature_dfs_clean`: List of cleaned training DataFrames.
- `Test_features_dfs_clean`: List of cleaned testing DataFrames.
- `MAX_CORR`: Maximum allowed correlation between features (default is 0.7).

### Returns:
- `feature_dfs_No_Corr`: List of training DataFrames with reduced multicollinearity.
- `Test_features_dfs_No_Corr`: List of testing DataFrames with correlated features removed.


In [ ]:
def remove_corr_values(feature_dfs_clean, Test_features_dfs_clean, MAX_CORR = 0.7):

    feature_df_clean = feature_dfs_clean[0].copy()

    # Set of features to always keep, even if highly correlated
    features_to_keep = {'mid_prices', 'Price Movement t+1', 'Price Movement t+5', 'Price Movement t+10'}

    # Compute the correlation matrix
    corr_matrix = feature_df_clean.corr()

    # Find feature pairs with high correlation (excluding self-correlation)
    high_corr_pairs = [(corr_matrix.index[x], corr_matrix.columns[y]) for x, y in zip(*
        (np.where((np.abs(corr_matrix) >= MAX_CORR) & (corr_matrix != 1.0))))]

    # Identify highly correlated features to remove, ensuring one feature remains from each pair
    features_to_remove = set()
    seen_features = set()

    for feature_x, feature_y in high_corr_pairs:
        if feature_x in features_to_keep or feature_y in features_to_keep:
            continue  # Always keep the essential features

        if feature_x not in seen_features:
            features_to_remove.add(feature_y)  # Keep feature_x, remove feature_y
            seen_features.add(feature_x)
        elif feature_y not in seen_features:
            features_to_remove.add(feature_x)  # Keep feature_y, remove feature_x
            seen_features.add(feature_y)

    # Store removed highly correlated features
    removed_high_corr_features = list(features_to_remove)

    # Drop highly correlated features from the dataset
    feature_df_No_Corr = feature_df_clean.drop(columns=removed_high_corr_features, errors='ignore')

    feature_dfs_No_Corr = feature_dfs_clean.copy()
    Test_features_dfs_No_Corr = Test_features_dfs_clean.copy()

    for df in feature_dfs_No_Corr:
        df.drop(columns=removed_high_corr_features, errors='ignore', inplace=True)

    for df in Test_features_dfs_No_Corr:
        df.drop(columns=removed_high_corr_features, errors='ignore', inplace=True)

    return feature_dfs_No_Corr, Test_features_dfs_No_Corr



## Group data to prepare to send to model `group_data_dfs`

Generates windowed datasets from input feature and movement DataFrames for use in time-series modeling.

### Functionality:
- Splits each DataFrame in `dfs` into overlapping windows of size `window`, with a step size defined by `displacement`.
- Each window contains sequences of feature values, preserving temporal order.
- Appends future movement labels from `movement_dfs` based on the last index in each window, for all columns that start with `'Price Movement'`.

### Parameters:
- `dfs`: List of feature DataFrames (one per asset or stock).
- `movement_dfs`: Corresponding list of movement DataFrames with target labels.
- `window`: Number of timesteps in each rolling window (default: 10).
- `displacement`: Step size between consecutive windows (default: 1).
- `threshold`: Optional threshold for movement (not used in this implementation but may be relevant for later classification).

### Returns:
- `all_dfs`: List of windowed DataFrames, each containing sequences of features and associated movement labels.


In [ ]:
def group_data_dfs(dfs, movement_dfs, window=10, displacement=1, threshold=0.001):

    all_dfs = []

    # Process each dataset and its corresponding movement DataFrame.
    for idx, df in enumerate(dfs):
        L = len(df)
        if L < window:
            all_dfs.append(pd.DataFrame())
            continue

        new_data = {}

        # Generate windows for each column in the feature DataFrame.
        for col in df.columns:
            windows_list = []
            i = 0
            while (i * displacement + window) <= L:
                w = df[col].iloc[i * displacement : i * displacement + window].tolist()
                windows_list.append(w)
                i += 1
            new_data[col] = windows_list

        num_windows = i  # Total number of windows generated

        # For each window, compute the last index using vectorized operations.
        last_indices = np.arange(num_windows) * displacement + window - 1

         # Automatically extract movement values for each column in mov_df that starts with 'Price Movement'
        mov_df = movement_dfs[idx]
        movement_cols = [col for col in mov_df.columns if col.startswith('Price Movement')]
        for col in movement_cols:
            new_data[col] = mov_df.iloc[last_indices][col].tolist()

        all_dfs.append(pd.DataFrame(new_data))

    return all_dfs

# Compute rolling average to prepare to send to models `group_data_avg_dfs`
## Alternative to the previous function

Creates windowed datasets where each feature is averaged over rolling windows, combining historical context with target price movement labels.

### Functionality:
- Slides a window of size `window` across each DataFrame in `dfs` with a step of `displacement`.
- Computes the average value of each feature within the window, instead of keeping full sequences.
- Extracts corresponding movement labels from `movement_dfs` for all columns beginning with `'Price Movement'`.

### Parameters:
- `dfs`: List of feature DataFrames.
- `movement_dfs`: List of DataFrames containing movement labels.
- `window`: Number of timesteps in each rolling window (default: 10).
- `displacement`: Step size between windows (default: 2).
- `threshold`: Threshold placeholder for potential filtering (unused in current implementation).

### Returns:
- `all_dfs`: List of DataFrames where each row contains the average of windowed features and corresponding movement labels.


In [ ]:
def group_data_avg_dfs(dfs, movement_dfs, window=10, displacement=2, threshold=0.001):

    all_dfs = []
    for idx, df in enumerate(dfs):
        L = len(df)
        # We need at least (window) + 1 rows so that the special window and its next value exist.
        if L < window + 1:
            all_dfs.append(pd.DataFrame())
            continue

        new_data = {}
        # Process every column in the DataFrame.
        for col in df.columns:
            averages_list = []

            # Build subsequent windows using the displacement value.
            i = 0
            while (i * displacement + window) < L:
                w = df[col].iloc[i * displacement : i * displacement + window].tolist()
                averages_list.append(sum(w) / len(w))  # Compute average of the window
                i += 1

            new_data[col] = averages_list

        # Get the movement columns
        num_windows = i  # Total number of windows generated

        # For each window, compute the last index using vectorized operations.
        last_indices = np.arange(num_windows) * displacement + window - 1

         # Automatically extract movement values for each column in mov_df that starts with 'Price Movement'
        mov_df = movement_dfs[idx]
        movement_cols = [col for col in mov_df.columns if col.startswith('Price Movement')]
        for col in movement_cols:
            new_data[col] = mov_df.iloc[last_indices][col].tolist()

        all_dfs.append(pd.DataFrame(new_data))

    return all_dfs


## Prepare Data for Loop `prepare_data_for_loop`
### Larger function that call the previous functions

Prepares training and testing datasets for model input through a full preprocessing pipeline.

### Functionality:
- Splits raw training and testing data by stock.
- Extracts and separates future price movement targets (`t+1`, `t+5`, `t+10`).
- Cleans features by removing zero-variance columns and highly correlated features.
- Constructs averaged rolling window features for model input.
- Optionally includes data visualization (commented out).
- Returns full and de-correlated feature sets in both raw and windowed form.

### Returns:
- `train_group_dfs_clean`, `test_group_dfs_clean`: Rolling window averaged training and testing sets.
- `train_group_dfs_no_corr`, `test_group_dfs_no_corr`: Same as above, but with correlated features removed.
- `train_dfs_clean`, `test_dfs_clean`: Cleaned base feature DataFrames (no zero-variance features).
- `train_dfs_no_corr`, `test_dfs_no_corr`: Cleaned DataFrames with correlated features removed.


In [ ]:
def get_reduced_df(df_list, VALUE_PERCENTAGE = 0.05):

    reduced_list = []

    for df in df_list:
        num_rows = max(1, int(len(df) * VALUE_PERCENTAGE))  # Ensure at least 1 row is kept
        reduced_df = df.iloc[:num_rows]  # Take the first num_rows rows
        reduced_list.append(reduced_df)

    return reduced_list

# Set window for rolling comulative sum
WINDOW = 30

def prepare_data_for_loop(train_data, test_data, preprocessing_path):

      # Send csv data that returns 2x array (train and test) of 6: one per stock and one for combination of all stocks
      train_stocks_split = get_stocks_old(train_data) #
      test_stocks_split = get_stocks_old(test_data) #

      print('\nData split processing was successful\n')

      # Get the features for each the training and testing data - Still an array of 6
      train_dfs_stocks, test_dfs_stocks = get_feature_dfs_data(train_stocks_split, test_stocks_split) #2

      print('\nFeatures were successfully extracted\n')

      # Extract 'Price Movement' columns from train and test datasets
      price_movement_train = [df[['Price Movement t+1', 'Price Movement t+5', 'Price Movement t+10']] for df in train_dfs_stocks]
      price_movement_test = [df[['Price Movement t+1', 'Price Movement t+5', 'Price Movement t+10']] for df in test_dfs_stocks]

      # Remove the extracted columns from the original datasets
      train_dfs_stocks = [df.drop(['Price Movement t+1', 'Price Movement t+5', 'Price Movement t+10'], axis=1) for df in train_dfs_stocks]
      test_dfs_stocks = [df.drop(['Price Movement t+1', 'Price Movement t+5', 'Price Movement t+10'], axis=1) for df in test_dfs_stocks]

      print('\nPrice Movement Extracted for t+1, t+5 and t+10\n')

      # # Clean the features data frame by removing features containing majoratively 0.0
      train_dfs_clean, test_dfs_clean = clean_data(train_dfs_stocks, test_dfs_stocks) #3

      # # Removes highly correlated features (keeps one of the pair)
      train_dfs_no_corr, test_dfs_no_corr = remove_corr_values(train_dfs_clean, test_dfs_clean) #4

      # Get window of information in data frames
      train_group_dfs_clean = group_data_avg_dfs(train_dfs_clean, price_movement_train)
      test_group_dfs_clean = group_data_avg_dfs(test_dfs_clean, price_movement_test)
      train_group_dfs_no_corr = group_data_avg_dfs(train_dfs_no_corr, price_movement_train)
      test_group_dfs_no_corr = group_data_avg_dfs(test_dfs_no_corr, price_movement_test)

      print('\nExtracted low correlation features sucessfully\n')

      # Plots
    #   plot_stocks(train_dfs_stocks, preprocessing_path)
    #   plot_corr_matrix(train_dfs_clean, preprocessing_path, True)
    #   plot_corr_matrix(train_dfs_no_corr, preprocessing_path)
    #   save_vif_analysis(train_dfs_stocks, preprocessing_path)

      print('\nPreprocessing successful\n')

      # TODO return only 5% of the data
      return train_group_dfs_clean, test_group_dfs_clean, train_group_dfs_no_corr, test_group_dfs_no_corr, train_dfs_clean, test_dfs_clean, train_dfs_no_corr, test_dfs_no_corr

# **Part 2: Forecasting Mid-Price Movements**

## Random Forest

Trains and evaluates a Random Forest classifier on windowed time-series data for multi-class mid-price movement prediction. Outputs performance metrics and visual comparison.

### Functionality:
- Preprocesses the input data by flattening windowed features and scaling them.
- Trains a `RandomForestClassifier` with 100 trees.
- Predicts movement classes for the test set.
- Evaluates model performance using Accuracy, F1 score (weighted), Cohen’s Kappa, and Precision.
- Saves metrics to a CSV file and plots the actual vs. predicted labels.

### Parameters:
- `train`: Training DataFrame with averaged or windowed features and a movement label.
- `test`: Testing DataFrame in the same format.
- `csv_file_path`: File path to append model evaluation results.
- `path_figures`: Folder where prediction plots will be saved.
- `tnn`: Identifier (e.g. label or time horizon name) written to the CSV for clarity.

### Returns:
- A list containing:
  - The trained Random Forest model.
  - Scaled training features (`X_train_scaled`).
  - Scaled test features (`X_test_scaled`).
  - Feature names from the original DataFrame (`train.columns`).


In [ ]:
def run_random_forest(train, test, csv_file_path, path_figures, tnn):
    def prepare_data(df):
        X = np.stack([np.stack(row) for row in df.iloc[:, :-1].values], axis=0)
        y = df.iloc[:, -1].values.astype(int)
        return X, y

    X_train, y_train = prepare_data(train)
    X_test, y_test = prepare_data(test)

    # Reshape to (samples, timesteps * features) for Random Forest
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)

    # Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_flat)
    X_test_scaled = scaler.transform(X_test_flat)

    # Random Forest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)

    # Predictions
    predictions_class = model.predict(X_test_scaled)

    # Metrics calculation
    accuracy = accuracy_score(y_test, predictions_class)
    f1 = f1_score(y_test, predictions_class, average='weighted')
    kappa = cohen_kappa_score(y_test, predictions_class)
    precision = precision_score(y_test, predictions_class, average='weighted')

    # Write metrics to CSV
    with open(csv_file_path, 'a') as file:
        file.write(f'Random_Forest {tnn},{accuracy},{f1},{kappa},{precision}\n')

    # Plotting results
    plt.figure(figsize=(10, 6))
    plt.plot(y_test, label='Actual', linestyle='dashed')
    plt.plot(predictions_class, label='Predicted', linestyle='solid')
    plt.xlabel('Sample Index')
    plt.ylabel('Class')
    plt.title('Random Forest Predictions vs Actual Classes')
    plt.legend()
    plt.savefig(os.path.join(path_figures, 'random_forest_classification.png'))
    plt.close()

    return [model, X_train_scaled, X_test_scaled, train.columns]

## LSTMClassifier Pipeline

Implements and evaluates an LSTM-based deep learning model for multi-class classification using time-series feature data from high-frequency trading.



### prepare_data

Preprocesses a windowed DataFrame into 3D format suitable for LSTM input.

#### Output:
- `X`: 3D NumPy array with shape `(samples, timesteps, features)`
- `y`: Class label array (integer encoded)



### LSTMClassifier

Custom TensorFlow model with:
- Two stacked LSTM layers (`return_sequences=True`)
- Dropout layers for regularization
- Dense layers for classification output (logits, no softmax)



### run_lstm

Trains and evaluates the LSTM model, saving metrics and prediction plots.

#### Functionality:
- Prepares data and ensures proper input shape
- Applies feature scaling using `StandardScaler`
- Compiles and trains the LSTM model
- Evaluates model with:
  - Accuracy
  - Weighted F1 Score
  - Cohen’s Kappa
  - Weighted Precision
- Appends results to a CSV file
- Saves a plot comparing actual and predicted class labels

#### Parameters:
- `train`, `test`: Preprocessed DataFrames with time-series features and target class
- `csv_file_path`: Path to output metrics
- `path_figures`: Directory to save the plot
- `tnn`: Identifier for the time horizon or run
- `epochs`: Number of training epochs (default: 20)
- `batch_size`: Batch size for training (default: 32)

#### Returns:
- List containing the trained model, training and test feature arrays, and column names


Code Inpired by Charrejee et al. (2021): [Link article](https://arxiv.org/pdf/2111.01137)

In [ ]:
# Data preparation: stack features and ensure 3D shape: (samples, timesteps, features)
def prepare_data(df):
    X = np.stack([np.array(row) for row in df.iloc[:, :-1].values], axis=0)
    y = df.iloc[:, -1].values.astype(int)

    # Ensure X has 3 dimensions: (samples, timesteps, features)
    if X.ndim == 2:
        X = np.expand_dims(X, axis=-1)
    return X, y

# LSTMClassifier implemented in TensorFlow
class LSTMClassifier(tf.keras.Model):
    def __init__(self, input_size, hidden_size1=256, hidden_size2=128, output_size=3):
        """
        input_size: number of features per timestep
        hidden_size1: number of units for the first LSTM layer
        hidden_size2: number of units for the second LSTM layer
        output_size: number of output classes
        """
        super(LSTMClassifier, self).__init__()
        # First LSTM layer; return sequences for further processing
        self.lstm1 = tf.keras.layers.LSTM(hidden_size1, return_sequences=True)
        self.dropout1 = tf.keras.layers.Dropout(0.2)
        # Second LSTM layer; we use return_sequences=True so we can extract the last time step explicitly
        self.lstm2 = tf.keras.layers.LSTM(hidden_size2, return_sequences=True)
        self.dropout2 = tf.keras.layers.Dropout(0.2)
        self.fc1 = tf.keras.layers.Dense(8, activation='relu')
        self.fc2 = tf.keras.layers.Dense(output_size)  # Logits; no activation here

    def call(self, x, training=False):
        x = self.lstm1(x, training=training)
        x = self.dropout1(x, training=training)
        x = self.lstm2(x, training=training)
        # Extract the output at the last timestep
        x = x[:, -1, :]
        x = self.dropout2(x, training=training)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# Training and evaluation function
def run_lstm(train, test, csv_file_path, path_figures, tnn, epochs=20, batch_size=32):
    # Prepare data from pandas DataFrames
    X_train, y_train = prepare_data(train)
    X_test, y_test = prepare_data(test)
    X_columns = train.columns[:-1]

    # Ensure correct shape: if the second dimension is larger than the third, transpose.
    if X_train.shape[1] > X_train.shape[2]:
        X_train = np.transpose(X_train, (0, 2, 1))
        X_test = np.transpose(X_test, (0, 2, 1))

    # Scale features (flatten the time dimension for scaling and reshape back)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

    # Build the model. The input_size is the number of features per timestep.
    model = LSTMClassifier(input_size=X_train_scaled.shape[2])

    # Compile the model with Adam optimizer and sparse categorical crossentropy loss (from logits)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    # Train the model using model.fit with validation on the test set
    history = model.fit(X_train_scaled, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(X_test_scaled, y_test),
                        verbose=2)

    # Obtain predictions on the test set
    logits = model.predict(X_test_scaled)
    predictions_class = np.argmax(logits, axis=1)

    # Calculate evaluation metrics using scikit-learn
    accuracy = accuracy_score(y_test, predictions_class)
    f1 = f1_score(y_test, predictions_class, average='weighted')
    kappa = cohen_kappa_score(y_test, predictions_class)
    precision = precision_score(y_test, predictions_class, average='weighted')

    # Append the metrics to the CSV file
    with open(csv_file_path, 'a') as file:
        file.write(f'LSTM {tnn},{accuracy},{f1},{kappa},{precision}\n')

    # Plot the actual vs. predicted classes
    plt.figure(figsize=(10, 6))
    plt.plot(y_test, label='Actual', linestyle='dashed')
    plt.plot(predictions_class, label='Predicted', linestyle='solid')
    plt.xlabel('Time Steps')
    plt.ylabel('Class')
    plt.title('LSTM Predictions vs Actual Classes')
    plt.legend()
    plt.savefig(os.path.join(path_figures, 'lstm_classification.png'))
    plt.close()

    return [model, X_train, X_test, X_columns]


## ResNet-LSTM Classifier

Implements a hybrid architecture combining residual convolutional layers (ResNet) with LSTM layers for enhanced sequence modeling. Designed for mid-price movement prediction from time-series limit order book features.


### ResNetLSTMClassifier (TensorFlow Model)

A custom deep learning model that:
- Applies **1D convolutional layers** with residual connections to extract robust local patterns across time.
- Passes extracted features through stacked **LSTM layers** to capture temporal dependencies.
- Ends with dense layers for multi-class classification output (via logits).

#### Architecture Overview:
- Conv1D → BatchNorm → ReLU
- Residual block(s) with skip connections
- LSTM layers (with `return_sequences`)
- Dropout + Dense layers
- Final dense layer without activation (logits)


### run_resnet_lstm

Trains and evaluates the ResNet-LSTM model using windowed and scaled time-series input.

#### Functionality:
- Prepares the 3D feature data `(samples, timesteps, features)`
- Applies feature scaling
- Compiles the model with Adam optimizer and sparse categorical crossentropy loss
- Trains the model with early stopping or fixed epochs
- Evaluates performance using:
  - Accuracy
  - Weighted F1 score
  - Cohen’s Kappa
  - Weighted Precision
- Saves evaluation metrics to CSV and comparison plots to PNG

#### Parameters:
- `train`, `test`: DataFrames with 3D windowed features and class labels
- `csv_file_path`: Output path for saving metrics
- `path_figures`: Directory to save prediction plot
- `tnn`: Run label (e.g., time horizon)
- `epochs`: Training epochs (default: 20)
- `batch_size`: Training batch size (default: 32)

#### Returns:
- Trained model
- Scaled train and test features
- Feature column names from the original data

Code Inpired by Jia et al. (2025): [Link article](https://arxiv.org/pdf/2312.01020)

In [ ]:
# Custom model equivalent to the PyTorch ResNLS
class ResNLS(tf.keras.Model):
    def __init__(self, input_dim, hidden_dim=64, output_dim=3):
        """
        input_dim: number of time steps (or original feature length)
        hidden_dim: number of filters/hidden units in CNN and LSTM
        output_dim: number of classes for classification
        """
        super(ResNLS, self).__init__()
        # Create a trainable scalar weight parameter
        self.weight = self.add_weight(name="weight",
                                      shape=(1,),
                                      initializer=tf.zeros_initializer(),
                                      trainable=True)

        # CNN Block
        self.conv1 = tf.keras.layers.Conv1D(filters=hidden_dim,
                                            kernel_size=3,
                                            strides=1,
                                            padding='same',
                                            activation='relu')
        self.bn1 = tf.keras.layers.BatchNormalization(epsilon=1e-5)
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.conv2 = tf.keras.layers.Conv1D(filters=hidden_dim,
                                            kernel_size=3,
                                            strides=1,
                                            padding='same',
                                            activation='relu')
        self.bn2 = tf.keras.layers.BatchNormalization(epsilon=1e-5)
        # Global average pooling over the time dimension (equivalent to AdaptiveAvgPool1d(1))
        self.global_pool = tf.keras.layers.GlobalAveragePooling1D()
        # Fully-connected layer to map CNN output back to input dimension (time steps)
        self.fc_cnn = tf.keras.layers.Dense(input_dim)

        # LSTM Block: return_state=True so we can use the last hidden state
        self.lstm = tf.keras.layers.LSTM(hidden_dim, return_state=True)
        self.linear = tf.keras.layers.Dense(output_dim)

    def call(self, x, training=False):
        # x is expected to be of shape (batch, sequence_length, channels)
        # CNN Block
        cnn = self.conv1(x)
        cnn = self.bn1(cnn, training=training)
        cnn = self.dropout(cnn, training=training)
        cnn = self.conv2(cnn)
        cnn = self.bn2(cnn, training=training)
        cnn = self.global_pool(cnn)  # shape becomes (batch, hidden_dim)
        cnn = self.fc_cnn(cnn)       # shape becomes (batch, input_dim)
        # Expand dimensions to create a "channel" dimension, now (batch, input_dim, 1)
        cnn = tf.expand_dims(cnn, axis=-1)

        # Residual connection: add original input and weighted CNN output.
        # Assumes that the original input x has one channel and sequence length equals input_dim.
        residuals = x + self.weight * cnn

        # LSTM Block: process the residual sequence.
        # Only the final hidden state is used for classification.
        _, state_h, _ = self.lstm(residuals)
        y_hat = self.linear(state_h)
        return y_hat

# Data preparation function adjusted for TensorFlow (channels-last)
def prepare_data(df):
    # Stack feature columns; assume last column is the target
    X = np.stack(df.iloc[:, :-1].values, axis=0)
    y = df.iloc[:, -1].values.astype(int)
    # Reshape to (batch, sequence_length, channels); here channels=1
    return X.reshape(X.shape[0], X.shape[1], 1), y

# Training and evaluation function
def run_resnet(train, test, csv_file_path, path_figures, tnn, epochs=50, batch_size=64):
    # Get column names (for potential future use, e.g., in SHAP analysis)
    X_train_columns = train.columns
    X_train, y_train = prepare_data(train)
    X_test, y_test = prepare_data(test)

    # Standardize the data
    scaler_X = StandardScaler()
    X_train = scaler_X.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
    X_test = scaler_X.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)

    print("Unique class labels in train:", np.unique(y_train))
    print("Unique class labels in test:", np.unique(y_test))

    num_classes = len(np.unique(y_train))
    # Instantiate the model.
    # Here, input_dim corresponds to the sequence length (number of features)
    model = ResNLS(input_dim=X_train.shape[1], output_dim=num_classes)

    # Compile the model with Adam optimizer and sparse categorical crossentropy loss.
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

    # Train the model using model.fit, including validation on test data.
    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        verbose=2)

    # Evaluate predictions on test set.
    logits = model(X_test, training=False)
    predictions = tf.argmax(logits, axis=1).numpy()
    y_test_np = y_test

    # Compute classification metrics using scikit-learn.
    accuracy = accuracy_score(y_test_np, predictions)
    f1 = f1_score(y_test_np, predictions, average='weighted')
    kappa = cohen_kappa_score(y_test_np, predictions)
    precision = precision_score(y_test_np, predictions, average='weighted')

    # Append results to the CSV file.
    with open(csv_file_path, 'a') as file:
        file.write(f'ResNet {tnn},{accuracy},{f1},{kappa},{precision}\n')

    # Plot actual vs. predicted class labels.
    plt.figure(figsize=(10, 6))
    plt.plot(y_test_np, label='Actual', linestyle='dashed')
    plt.plot(predictions, label='Predicted', linestyle='solid')
    plt.xlabel('Sample Index')
    plt.ylabel('Class')
    plt.title('ResNet Classification Predictions vs Actual Classes')
    plt.legend()
    plt.savefig(os.path.join(path_figures, 'resnet_classification.png'))
    plt.close()

    print('SHAP CALLING')
    # shap(model, X_train, X_test, X_train_columns)
    print('SHAP DONE')

    return [model, X_train, X_test, X_train_columns]


## ARIMAX

Fits and evaluates an ARIMAX (AutoRegressive Integrated Moving Average with Exogenous inputs) model for time-series regression using both autoregressive and feature-driven components.

### Functionality:
- Splits the dataset into target (`y`) and exogenous variables (`X`).
- Scales the exogenous variables using `StandardScaler`.
- Fits an ARIMAX model on the training set using the specified order (default: (1, 1, 1)).
- Predicts values on the test set using the scaled exogenous features.
- Computes both regression and classification metrics:
  - Accuracy (based on rounded class predictions)
  - Mean Squared Error (MSE)
  - Mean Absolute Error (MAE)
  - R-squared (R²)
- Appends results to a CSV file and saves a plot comparing predicted and actual values.

### Parameters:
- `train`, `test`: DataFrames where the first column is the target and the remaining are features.
- `csv_file_path`: Path to append evaluation metrics.
- `path_figures`: Directory to save prediction plot.
- `tnn`: Identifier for model tracking (e.g. time horizon).
- `order`: ARIMA order tuple (p, d, q) (default: (1, 1, 1)).

### Returns:
- List containing:
  - Trained SARIMAX model
  - Scaled training and testing features
  - Raw training and testing target arrays
  - Feature column names


In [ ]:
def run_arimax(train, test, csv_file_path, path_figures, tnn, order=(1, 1, 1)):
    warnings.filterwarnings("ignore")  # Suppress convergence warnings

    # Split into target and features
    y_train, X_train = train.iloc[:, 0], train.iloc[:, 1:]
    y_test, X_test = test.iloc[:, 0], test.iloc[:, 1:]

    # Scale exogenous variables
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Fit ARIMAX model with simpler order to reduce overfitting
    model = SARIMAX(endog=y_train, exog=X_train_scaled, order=order,
                    enforce_stationarity=True, enforce_invertibility=True)
    model_fit = model.fit(disp=False, maxiter=300, method='powell')

    # Forecast
    predictions = model_fit.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1, exog=X_test_scaled)

    # Round for classification-style accuracy
    y_pred_classes = np.round(predictions).astype(int)
    y_true_classes = np.round(y_test).astype(int)

    # Metrics
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    metrics = [accuracy, mse, mae, r2]

    # Log metrics
    with open(csv_file_path, 'a') as file:
        file.write(f'arimax{tnn},{" ,".join(map(str, metrics))}\n')

    # Plot predictions
    plt.figure(figsize=(10, 6))
    plt.plot(y_test.values, label='Actual', linestyle='dashed')
    plt.plot(predictions, label='Predicted', linestyle='solid')
    plt.xlabel('Sample Index')
    plt.ylabel('Target')
    plt.title(f'ARIMAX Predictions vs Actual (Model {tnn})')
    plt.legend()
    plt.savefig(os.path.join(path_figures, f'arimax_prediction_{tnn}.png'))
    plt.close()

    return [model_fit, X_train, X_test, y_train, y_test, X_train.columns]


# Function to get Absolute Mean SHAP Values
It is recommended to run a specific model and with a specific time horizong as they take a lot of computing power and time

In [ ]:
def get_shap_arimax(model_fit, X_train, X_test, y_train, y_test, feature_names, file_path, order=(1, 1, 1), z=1, background_size=50, EXPLAIN=2, ROUNDING=4):
    # Create a scaler and fit on the training exogenous data (unscaled)
    scaler = StandardScaler()
    scaler.fit(X_train)

    # Define a prediction wrapper function that maps unscaled exogenous features to predictions.
    def arimax_predict_wrapper(X):
        # X is expected to be a 2D numpy array (samples, features)
        preds = []
        for i in range(X.shape[0]):
            x_row = X[i].reshape(1, -1)
            # Scale the input row using the already fitted scaler.
            x_scaled = scaler.transform(x_row)
            # Forecast one step ahead using the ARIMAX model.
            pred = model_fit.forecast(steps=1, exog=x_scaled)
            # --- Amendment: Use .iloc[0] instead of pred[0] to extract the first value
            preds.append(pred.iloc[0])
            # --- End Amendment
        return np.array(preds)

    # Prepare background data from the training exogenous features.
    X_train_array = X_train.values
    if X_train_array.shape[0] > background_size:
        indices = np.random.choice(X_train_array.shape[0], background_size, replace=False)
        background_data = X_train_array[indices]
    else:
        background_data = X_train_array

    # Initialize SHAP KernelExplainer using the ARIMAX prediction wrapper.
    explainer = shap.KernelExplainer(arimax_predict_wrapper, background_data, feature_names=list(feature_names))

    # Compute SHAP values for the first EXPLAIN samples from the test set.
    X_test_array = X_test.iloc[:EXPLAIN].values
    shap_values = explainer.shap_values(X_test_array)

    # Calculate the mean absolute SHAP value for each feature.
    mean_shap_abs_values = np.round(np.mean(np.abs(shap_values), axis=0), ROUNDING)

    # Create a DataFrame to display feature importance.
    df_shap = pd.DataFrame({
        "Feature": list(feature_names),
        "Average": mean_shap_abs_values
    })
    df_shap.sort_values(by="Average", ascending=False, inplace=True)

    # Save the SHAP values DataFrame to CSV in the figures folder.
    if file_path:
        os.makedirs(file_path, exist_ok=True)
        df_shap.to_csv(os.path.join(file_path, f'ARIMAX_stock_{z}.csv'), index=False)
        print(f"Saved SHAP values to {file_path}")
    else:
        print("No file path provided; CSV not saved.")
    return shap_values


In [ ]:
def get_shap_random_forest(model, X_train_scaled, X_test_scaled, feature_names, z=1, file_path='', EXPLAIN=20, ROUNDING=4):

    # Ensure the feature_names list matches the number of features in X_train_scaled.
    n_features = X_train_scaled.shape[1]
    if len(feature_names) != n_features:
        print(f"Warning: feature_names length {len(feature_names)} does not match number of features {n_features}. Truncating.")
        feature_names = list(feature_names)[:n_features]

    # Initialize TreeExplainer with the trained Random Forest model.
    explainer = shap.TreeExplainer(model)

    # Compute SHAP values for the first EXPLAIN samples from the test set.
    X_test_explain = X_test_scaled[:EXPLAIN]
    shap_values = explainer.shap_values(X_test_explain)

    # Compute overall mean absolute SHAP importance
    if isinstance(shap_values, list):
        if len(shap_values) > 1:
            # Multi-class: for each class compute mean absolute values (shape: (n_features,))
            mean_shap_class = [np.mean(np.abs(sv), axis=0) for sv in shap_values]
            mean_shap_class = [np.round(arr, ROUNDING) for arr in mean_shap_class]
            # Compute overall average importance by averaging across classes
            overall_importance = np.round(np.mean(mean_shap_class, axis=0), ROUNDING)
        else:
            overall_importance = np.round(np.mean(np.abs(shap_values[0]), axis=0), ROUNDING)
    else:
        overall_importance = np.round(np.mean(np.abs(shap_values), axis=0), ROUNDING)

    # Ensure overall_importance is 1D
    overall_importance = np.asarray(overall_importance).flatten()

    # Ensure feature_names is a list with the same length as the number of features
    n_features = overall_importance.shape[0]
    if len(feature_names) != n_features:
        feature_names = list(feature_names)[:n_features]

    mean_shap_abs_values = np.round(np.abs(shap_values).mean(axis=0), ROUNDING)
    shap_avg = np.mean(mean_shap_abs_values, axis=1)


    # Multi-class: assume 3 classes and create a DataFrame with per-class and overall averages.
    # mean_shap_class = [np.round(np.mean(np.abs(sv), axis=0), ROUNDING).flatten() for sv in shap_values]
    # print(mean_shap_class)

    df_shap = pd.DataFrame({
        "Feature": feature_names,
        "Increase": mean_shap_abs_values[:, 0],
        "No Movement": mean_shap_abs_values[:, 1],
        "Decrease": mean_shap_abs_values[:, 2],
        "Average": shap_avg
    })
    df_shap.sort_values(by="Average", ascending=False, inplace=True)


    # Save the DataFrame to CSV if a file path is provided
    if file_path:

        df_shap.to_csv(os.path.join(file_path, f'Shap_Random_Forest_stock_{z}.csv'), index=False)
        print(f"Saved SHAP values to {file_path}")
    else:
        print("No file path provided; CSV not saved.")


    return shap_values


In [ ]:
def get_shap_resnet(model, X_train_scaled, X_test_scaled, feature_names, z,
                    file_path='results/T1/stock_0/figures', background_size=50, EXPLAIN=20, ROUNDING=4):
    # Convert 3D data (samples, features, channel) to 2D (samples, features)
    X_train_2d = np.squeeze(X_train_scaled, axis=-1)
    X_test_2d  = np.squeeze(X_test_scaled, axis=-1)

    # Define a prediction wrapper that expands 2D input back to 3D for your model.
    def model_predict_wrapper(X):
        X_expanded = np.expand_dims(X, axis=-1)
        return model.predict(X_expanded)

    # Prepare background data: select a random subset from the training data
    if X_train_2d.shape[0] > background_size:
        indices = np.random.choice(X_train_2d.shape[0], background_size, replace=False)
        background_data = X_train_2d[indices]
    else:
        background_data = X_train_2d

    # Create an explicit masker using the background data.
    masker = shap.maskers.Independent(background_data)

    # Initialize the SHAP explainer with the prediction wrapper and masker.
    explainer = shap.Explainer(model_predict_wrapper, masker, feature_names=feature_names)

    # Compute SHAP values for the first EXPLAIN samples from the test set.
    shap_values = explainer.shap_values(X_test_2d[:EXPLAIN])

    # Compute mean absolute SHAP values per feature
    # Handle multi-class case if shap_values is a list
    if isinstance(shap_values, list):
        if len(shap_values) > 1:
            # Multi-class: stack each class's mean absolute SHAP values (resulting shape: (n_features, n_classes))
            mean_shap_abs_values = np.stack(
                [np.round(np.mean(np.abs(sv), axis=0), ROUNDING) for sv in shap_values], axis=1
            )
        else:
            # Single class but provided in a list; expand dims to mimic 2D array
            mean_shap_abs_values = np.round(np.mean(np.abs(shap_values[0]), axis=0), ROUNDING)
            mean_shap_abs_values = mean_shap_abs_values[:, np.newaxis]
    else:
        # In case shap_values is not a list
        mean_shap_abs_values = np.round(np.mean(np.abs(shap_values), axis=0), ROUNDING)
        if mean_shap_abs_values.ndim == 1:
            mean_shap_abs_values = mean_shap_abs_values[:, np.newaxis]

    # Compute overall importance by averaging across classes
    overall_importance = np.round(np.mean(mean_shap_abs_values, axis=1), ROUNDING)
    overall_importance = np.asarray(overall_importance).flatten()

    # Ensure feature_names is a list with the same length as the number of features
    n_features = overall_importance.shape[0]
    if len(feature_names) != n_features:
        feature_names = list(feature_names)[:n_features]

    # Compute an average SHAP value per feature across classes
    shap_avg = np.mean(mean_shap_abs_values, axis=1)

    # Create DataFrame.
    # If we have exactly 3 classes, use the following column names.
    if mean_shap_abs_values.shape[1] == 3:
        df_shap = pd.DataFrame({
            "Feature": feature_names,
            "Increase": mean_shap_abs_values[:, 0],
            "No Movement": mean_shap_abs_values[:, 1],
            "Decrease": mean_shap_abs_values[:, 2],
            "Average": shap_avg
        })
    else:
        # For other numbers of classes, name the columns generically.
        class_columns = [f"Class_{i}" for i in range(mean_shap_abs_values.shape[1])]
        data = {"Feature": feature_names, "Average": shap_avg}
        for i, col in enumerate(class_columns):
            data[col] = mean_shap_abs_values[:, i]
        df_shap = pd.DataFrame(data)

    df_shap.sort_values(by="Average", ascending=False, inplace=True)

    # Save the DataFrame to CSV if a file path is provided
    if file_path:
        os.makedirs(file_path, exist_ok=True)
        save_path = os.path.join(file_path, f'ResNet Stock {z}.csv')
        df_shap.to_csv(save_path, index=False)
        print(f"Saved SHAP values to {save_path}")
    else:
        print("No file path provided; CSV not saved.")

    return shap_values


In [ ]:
def get_shap_lstm(model, X_train_scaled, X_test_scaled, feature_names, z = 1, file_path='', background_size=50, EXPLAIN=20, ROUNDING=4):
    # For run_lstm, data is expected to have shape (n_samples, 1, n_features).
    # Squeeze the timestep dimension (axis=1) to obtain 2D arrays: (n_samples, n_features)
    X_train_2d = np.squeeze(X_train_scaled, axis=1)
    X_test_2d  = np.squeeze(X_test_scaled, axis=1)

    # Define a prediction wrapper that adds the timestep dimension back (to shape: (batch, 1, n_features))
    def model_predict_wrapper(X):
        X_expanded = np.expand_dims(X, axis=1)
        return model.predict(X_expanded)

    # Prepare background data: randomly select from training data (2D)
    if X_train_2d.shape[0] > background_size:
        indices = np.random.choice(X_train_2d.shape[0], background_size, replace=False)
        background_data = X_train_2d[indices]
    else:
        background_data = X_train_2d

    # Create an explicit masker using the background data
    masker = shap.maskers.Independent(background_data)

    # Initialize the SHAP explainer with the prediction wrapper and masker
    explainer = shap.Explainer(model_predict_wrapper, masker, feature_names=feature_names)

    # Compute SHAP values for the first EXPLAIN samples from the test set (2D)
    shap_values = explainer.shap_values(X_test_2d[:EXPLAIN])

    # Compute overall mean absolute SHAP importance
    if isinstance(shap_values, list):
        if len(shap_values) > 1:
            # Multi-class: for each class compute mean absolute values (shape: (n_features,))
            mean_shap_class = [np.mean(np.abs(sv), axis=0) for sv in shap_values]
            mean_shap_class = [np.round(arr, ROUNDING) for arr in mean_shap_class]
            # Compute overall average importance by averaging across classes
            overall_importance = np.round(np.mean(mean_shap_class, axis=0), ROUNDING)
        else:
            overall_importance = np.round(np.mean(np.abs(shap_values[0]), axis=0), ROUNDING)
    else:
        overall_importance = np.round(np.mean(np.abs(shap_values), axis=0), ROUNDING)

    # Ensure overall_importance is 1D
    overall_importance = np.asarray(overall_importance).flatten()

    # Ensure feature_names is a list with the same length as the number of features
    n_features = overall_importance.shape[0]
    if len(feature_names) != n_features:
        feature_names = list(feature_names)[:n_features]

    mean_shap_abs_values = np.round(np.abs(shap_values).mean(axis=0), ROUNDING)
    shap_avg = np.mean(mean_shap_abs_values, axis=1)


    # Multi-class: assume 3 classes and create a DataFrame with per-class and overall averages.
    # mean_shap_class = [np.round(np.mean(np.abs(sv), axis=0), ROUNDING).flatten() for sv in shap_values]
    # print(mean_shap_class)

    df_shap = pd.DataFrame({
        "Feature": feature_names,
        "Increase": mean_shap_abs_values[:, 0],
        "No Movement": mean_shap_abs_values[:, 1],
        "Decrease": mean_shap_abs_values[:, 2],
        "Average": shap_avg
    })
    df_shap.sort_values(by="Average", ascending=False, inplace=True)


    # Save the DataFrame to CSV if a file path is provided
    if file_path:

        df_shap.to_csv(os.path.join(file_path, f'Shap_LSTM_stock_{z}.csv'), index=False)
        print(f"Saved SHAP values to {file_path}")
    else:
        print("No file path provided; CSV not saved.")


    return shap_values

# **Loop to Run Entire Pipeline**

## ⚠️ Warning: This action will overwrite existing files.

Main execution pipeline for end-to-end model training, evaluation, and result aggregation across multiple stock datasets.

### Functionality:
- Iterates over each folder in the `data/` directory (each representing a dataset).
- Creates a full preprocessing folder structure, including:
  - `Correlation_Matrix_Clean`
  - `Correlation_Matrix_No_Corr`
  - `Mid_Price_Graph`
  - `VIF`
- Loads and processes `train.csv` and `test.csv` files from each folder.
- Cleans and prepares data using:
  - Feature cleaning
  - Correlation filtering
  - Rolling window averaging
- Optionally reduces data size or removes corrupt first rows.
- Trains and evaluates multiple models per stock and per target horizon:
  - Random Forest
  - K-Means Clustering
  - ARIMAX
  - LSTM
  - ResNet
- Runs all models for three time horizons: `t+1`, `t+5`, and `t+10`.

### Output:
- Creates subfolders for each stock (`stock_0`, `stock_1`, ...) containing:
  - A `results.csv` file with evaluation metrics per model
  - A `figures/` folder with prediction plots
- Aggregates all model results into a final `all_results.csv` sorted by accuracy.

### Parameters:
- `path_data` (default: `'data'`): Path to the top-level directory containing folders with `train.csv` and `test.csv` for each stock dataset.

### Notes:
- Automatically deletes and recreates the `Preprocessing/` directory if it already exists.
- Logs errors per target if any model fails during training or prediction.
- Model-specific SHAP visualisation code is included but currently commented out.


In [ ]:
def run_pipeline(path_data = 'data'):

    for folder_name in os.listdir(path_data):
        folder_path = os.path.join(path_data, folder_name)
        if os.path.isdir(folder_path):

    ###################################################################################################################################
    #
    #                                                 CREATE FOLDERS FOR PREPROCESSING DATA
    #
    ###################################################################################################################################

            path = os.path.join(folder_name, 'results')

            # Define the full path for the "Preprocessing" folder
            preprocessing_path = os.path.join(path, 'Preprocessing')

            print(f"Processing folder: {preprocessing_path}")
            # If the "Preprocessing" folder exists, remove it along with its contents

            if os.path.exists(preprocessing_path):
                shutil.rmtree(preprocessing_path)
                print(f"Deleted existing folder and its contents: {preprocessing_path}")

            # Create the "Preprocessing" folder
            os.makedirs(preprocessing_path, exist_ok=True)
            print(f"Created folder: {preprocessing_path}")

            # List of subfolder names to create inside "Preprocessing"
            subfolders = [
                'Correlation_Matrix_Clean',
                'Correlation_Matrix_No_Corr',
                'Mid_Price_Graph',
                'VIF'
            ]

            # Create each subfolder
            for subfolder in subfolders:
                subfolder_path = os.path.join(preprocessing_path, subfolder)
                os.makedirs(subfolder_path, exist_ok=True)
                print(f"Created folder: {subfolder_path}")

    ###################################################################################################################################
    #
    #                                                 GATHER DATA TO RUN IN ML MODELS
    #
    ###################################################################################################################################


            print(f"Processing folder: {folder_path}")
            train_data = pd.read_csv(os.path.join(folder_path, 'train.csv'), delimiter=';')
            test_data = pd.read_csv(os.path.join(folder_path, 'test.csv'), delimiter=';')
            train_group_clean, test_group_clean, train_group_no_corr, test_group_no_corr, train_clean, test_clean, train_no_corr, test_no_corr = prepare_data_for_loop(train_data.T, test_data.T, preprocessing_path)

            # TODO CHANGE Drop first line of the dataframes because np float 64 appears in the first line
            train_group_clean = [df.drop(df.index[0]) for df in train_group_clean]
            test_group_clean = [df.drop(df.index[0]) for df in test_group_clean]
            train_group_no_corr = [df.drop(df.index[0]) for df in train_group_no_corr]
            test_group_no_corr = [df.drop(df.index[0]) for df in test_group_no_corr]


            # Used for debugging by running reduced set of data
            # All models excluding ARIMAX
            # train_group_clean = get_reduced_df(train_group_clean)
            # test_group_clean = get_reduced_df(test_group_clean)
            # train_group_no_corr = get_reduced_df(train_group_no_corr)
            # test_group_no_corr = get_reduced_df(test_group_no_corr)

            # Delete all folders in T1 first
            # if os.path.exists(path):
            #     for folder in os.listdir(path):
            #         folder_path = os.path.join(path, folder)
            #         print(folder_path)
            #         if os.path.isdir(folder_path):
            #             shutil.rmtree(folder_path)

            # Loop stock
            for i in range(len(train_clean)):
                print(f"\nProcessing stock: {i}\n")
                # Create a folder for the current stock
                path_results = os.path.join(path, f'stock_{i}')
                os.makedirs(path_results, exist_ok=True)


                # Create folders for results and figures
                path_figures = os.path.join(path_results, 'figures')
                os.makedirs(path_figures, exist_ok=True)

                # Create a CSV file path directly under stock folder
                csv_file_path = os.path.join(path_results, 'results.csv')
                with open(csv_file_path, 'w') as file:
                    pass  # Empty CSV file creation

                train = train_group_clean[i]
                test = test_group_clean[i]

                lst = ['Price Movement t+1','Price Movement t+5','Price Movement t+10']
                # Run instant +1, +5, +10
                # Loop through each column in the list
                for tn in lst:
                    use_train = train.copy()  # Copy original DataFrame
                    use_test = test.copy()  # Copy original DataFrame
                    use_train.drop(columns=[col for col in lst if col != tn], inplace=True)
                    use_test.drop(columns=[col for col in lst if col != tn], inplace=True)

                    # use_train = use_train[[tn]]  # Keep only the column in tn
                    trainn = use_train  # Store the filtered DataFrame in trainn
                    testt = use_test  # Store the filtered DataFrame in testt

                    try:
                        # run_linear_regression(trainn, testt, csv_file_path, path_figures)
                        Random_Forest_data = run_random_forest(trainn, testt, csv_file_path, path_figures, tn)
                        Kclustering_data = run_kclustering(trainn, testt, csv_file_path, path_figures, tn)
                        ARIMA_data = run_arimax(trainn, testt, csv_file_path, path_figures, tn)
                        LSTM_data = run_lstm(trainn, testt, csv_file_path, path_figures, tn)
                        ResNet_data = run_resnet(trainn, testt, csv_file_path, path_figures, tn)

                        ## Get SHAP values
                        # csv_file_path = os.path.join(path_results, 'results.csv')
                        # with open(csv_file_path, 'w') as file:
                        #     pass  # Empty CSV file creation
                        # get_shap_resnet(ResNet_data[0], ResNet_data[1], ResNet_data[2], ResNet_data[3])
                        # get_shap(LSTM_data[0], LSTM_data[1], LSTM_data[2], LSTM_data[3])
                    except Exception as e:
                        print('##############################################')
                        print('##############################################')
                        print(f"Error processing {tn}: {e}")
                        print('##############################################')
                        print('##############################################')
                        continue

            all_rows = []

            # Iterate through stock folders
            for stock_folder in os.listdir(path):
                stock_path = os.path.join(path, stock_folder)
                if os.path.isdir(stock_path):
                    csv_file = os.path.join(stock_path, 'results.csv')
                    if os.path.exists(csv_file):
                        df = pd.read_csv(csv_file,header=None)
                        for _, row in df.iterrows():
                            result_row = {
                                'stock_number': stock_folder,
                                'model': row[0],
                                'accuracy': float(row[1]),
                                'f1_score': float(row[2]),
                                'kappa': float(row[3]),
                                'precision': float(row[4])
                            }
                            all_rows.append(result_row)

            ## Combine all rows into a DataFrame
            if all_rows:
                final_df = pd.DataFrame(all_rows)
                final_df = final_df.sort_values(by='accuracy', ascending=False)

                final_csv_path = os.path.join(path, 'all_results.csv')
                final_df.to_csv(final_csv_path, index=False)
                print(f"All results combined and sorted by accuracy into {final_csv_path}")
            else:
                print("No results.csv files found.")
    return


# Generate Summary Statistics

In [ ]:
# results_df = pd.read_csv('model_results/model_kfold_results.csv')
# results_df.head()

# summary = results_df.groupby('model')['rmse'].agg(['mean', 'std']).reset_index()
# print("\nSummary statistics (average RMSE and standard deviation):")
# print(summary)

# plt.figure(figsize=(10, 6))
# plt.bar(summary['model'], summary['mean'], yerr=summary['std'], capsize=5, color='skyblue')
# plt.xlabel('Model')
# plt.ylabel('Average RMSE')
# plt.title('Average RMSE per Model (with Error Bars)')
# plt.tight_layout()
# plt.show()

# plt.figure(figsize=(10, 6))
# results_df.boxplot(column='rmse', by='model', grid=False)
# plt.xlabel('Model')
# plt.ylabel('RMSE')
# plt.title('RMSE Distribution per Model')
# plt.suptitle('')
# plt.tight_layout()
# plt.show()

# best_model = summary.loc[summary['mean'].idxmin()]
# print(f"\nBest model: {best_model['model']} with average RMSE of {best_model['mean']:.4f} (std={best_model['std']:.4f})")


## Sectional run of the progam
Run Section of the code with VS Code or Google Colab below

In [ ]:
# folder_path = '/content/drive/MyDrive/Dissertation/Zscore'

# train_data = pd.read_csv(os.path.join(folder_path, 'train.csv'), delimiter=';', header=None)
# test_data = pd.read_csv(os.path.join(folder_path, 'test.csv'), delimiter=';', header=None)

# train_group_clean, test_group_clean, train_group_no_corr, test_group_no_corr, train_clean, test_clean, train_no_corr, test_no_corr = prepare_data_for_loop(train_data.T, test_data.T)



# for folder_name in os.listdir(path):
#     folder_path = os.path.join(path, folder_name)
#     if os.path.isdir(folder_path):
#         print(f"Processing folder: {folder_path}")
#         train_data = pd.read_csv(os.path.join(folder_path, 'train.csv'))
#         test_data = pd.read_csv(os.path.join(folder_path, 'test.csv'))
#         train_clean, test_clean, train_no_corr, test_no_corr = prepare_data_for_loop(train_data.T, test_data.T)
#         break
#         # Add your processing code here
# #

## Get SHAP manually
Below is the iteration used to fish out the SHAP values

In [ ]:
# folder_path = 'Data\T1'
# train_data = pd.read_csv(os.path.join(folder_path, 'train.csv'), delimiter=';')
# test_data = pd.read_csv(os.path.join(folder_path, 'test.csv'), delimiter=';')
# train_group_clean, test_group_clean, train_group_no_corr, test_group_no_corr, train_clean, test_clean, train_no_corr, test_no_corr = prepare_data_for_loop(train_data.T, test_data.T, ' ')

# train_group_clean = get_reduced_df(train_group_clean)
# test_group_clean = get_reduced_df(test_group_clean)
# train_group_no_corr = get_reduced_df(train_group_no_corr)
# test_group_no_corr = get_reduced_df(test_group_no_corr)

In [ ]:

# path_temp = 'temp'
# csv_file_path_temp = os.path.join(path_temp, 'results.csv')
# with open(csv_file_path_temp, 'w') as file:
#     pass  # Empty CSV file creation

# tr = train_group_clean[0]
# te = test_group_clean[0]
# lst = ['Price Movement t+1','Price Movement t+5']
# # Run instant +1, +5, +10
# # Loop through each column in the list

# trr = tr.copy()  # Copy original DataFrame
# tee = te.copy()  # Copy original DataFrame
# trr.drop(columns=[col for col in lst], inplace=True)
# tee.drop(columns=[col for col in lst], inplace=True)

# a = run_arimax(trr, tee, csv_file_path_temp , 'temp', 'Price Movement t+10')

## FUNCTION NOT USED IN CODE - CAN BE ADDED - Runs Kclustering

Applies K-Means clustering to classify mid-price movement based on flattened, windowed feature data. Evaluates clustering performance against actual class labels.

### Functionality:
- Prepares time-series windowed data by flattening and scaling features.
- Fits a `KMeans` clustering model on the training set using a specified number of clusters.
- Assigns cluster labels to the test set and compares them to actual class labels.
- Calculates evaluation metrics: Accuracy, F1 score (weighted), Cohen’s Kappa, and Precision.
- Writes performance metrics to a CSV file and saves a prediction vs. actual comparison plot.

### Parameters:
- `train`: Training DataFrame with windowed or averaged features and a movement label.
- `test`: Testing DataFrame with the same format.
- `csv_file_path`: Path to the CSV file where evaluation results are saved.
- `path_figures`: Directory to save the plot comparing predicted clusters and actual classes.
- `tnn`: Identifier for tracking the time horizon or dataset (used in CSV output).
- `n_clusters`: Number of clusters for K-Means (default: 3).

### Output:
- Appends performance metrics to a CSV file.
- Saves a PNG plot showing predicted cluster labels vs actual class labels.


In [ ]:
def run_kclustering(train, test, csv_file_path, path_figures, tnn, n_clusters=3):
    def prepare_data(df):
        X = np.stack([np.stack(row) for row in df.iloc[:, :-1].values], axis=0)
        y = df.iloc[:, -1].values.astype(int)
        return X, y

    X_train, y_train = prepare_data(train)
    X_test, y_test = prepare_data(test)

    # Reshape to (samples, timesteps * features) for KMeans clustering
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)

    # Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_flat)
    X_test_scaled = scaler.transform(X_test_flat)

    # KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(X_train_scaled)

    # Predictions
    predictions_class = kmeans.predict(X_test_scaled)

    # Metrics calculation
    accuracy = accuracy_score(y_test, predictions_class)
    f1 = f1_score(y_test, predictions_class, average='weighted')
    kappa = cohen_kappa_score(y_test, predictions_class)
    precision = precision_score(y_test, predictions_class, average='weighted')

    # Write metrics to CSV
    with open(csv_file_path, 'a') as file:
        file.write(f'kclustering {tnn},{accuracy},{f1},{kappa},{precision}\n')

    # Plotting results
    plt.figure(figsize=(10, 6))
    plt.plot(y_test, label='Actual', linestyle='dashed')
    plt.plot(predictions_class, label='Predicted (Clusters)', linestyle='solid')
    plt.xlabel('Sample Index')
    plt.ylabel('Cluster/Class')
    plt.title('KMeans Clustering Predictions vs Actual Classes')
    plt.legend()
    plt.savefig(os.path.join(path_figures, 'kmeans_clustering.png'))
    plt.close()


## FUNCTION NOT USED IN CODE - CAN BE ADDED - Runs linear regression

Trains and evaluates a linear regression model for multi-class classification using windowed feature data. Outputs model performance metrics and a comparison plot.

### Functionality:
- Prepares windowed feature data by flattening it into 2D format suitable for linear regression.
- Scales the input features using `StandardScaler`.
- Trains a `LinearRegression` model on the processed training data.
- Predicts class values for the test set and rounds predictions to nearest class (0, 1, or 2).
- Calculates classification metrics: Accuracy, F1 score (weighted), Cohen’s Kappa, and Precision.
- Appends results to a CSV file.
- Saves a plot comparing predicted vs. actual class labels.

### Parameters:
- `train`: Preprocessed training DataFrame with windowed features and a single movement label column.
- `test`: Preprocessed testing DataFrame in the same format.
- `csv_file_path`: File path for storing model evaluation metrics.
- `path_figures`: Directory path to save the prediction comparison plot.

### Output:
- Appends a row of performance metrics to the specified CSV file.
- Saves a PNG plot comparing actual vs. predicted class labels.


In [ ]:
def run_linear_regression(train, test, csv_file_path, path_figures):
    def prepare_data(df):
        X = np.stack([np.stack(row) for row in df.iloc[:, :-1].values], axis=0)
        y = df.iloc[:, -1].values.astype(int)
        return X, y

    X_train, y_train = prepare_data(train)
    X_test, y_test = prepare_data(test)

    # Reshape to (samples, timesteps * features) for linear regression
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)

    # Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_flat)
    X_test_scaled = scaler.transform(X_test_flat)

    # Linear regression model
    model = LinearRegression()
    model.fit(X_train_scaled, y_train)

    # Predictions
    predictions = model.predict(X_test_scaled)
    predictions_class = np.clip(np.round(predictions), 0, 2).astype(int)  # Ensure predictions are valid classes (0,1,2)

    # Metrics calculation
    accuracy = accuracy_score(y_test, predictions_class)
    f1 = f1_score(y_test, predictions_class, average='weighted')
    kappa = cohen_kappa_score(y_test, predictions_class)
    precision = precision_score(y_test, predictions_class, average='weighted')

    # Write metrics to CSV
    with open(csv_file_path, 'a') as file:
        file.write(f'linear_regression,{accuracy},{f1},{kappa},{precision}\n')

    # Plotting results
    plt.figure(figsize=(10, 6))
    plt.plot(y_test, label='Actual', linestyle='dashed')
    plt.plot(predictions_class, label='Predicted', linestyle='solid')
    plt.xlabel('Sample Index')
    plt.ylabel('Class')
    plt.title('Linear Regression Predictions vs Actual Classes')
    plt.legend()
    plt.savefig(os.path.join(path_figures, 'linear_regression_classification.png'))
    plt.close()